In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

In [ ]:
env OPENAI_API_KEY=''

In [ ]:
import openai

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00


# Load Dataset



In [ ]:
from datasets import load_dataset
ds = load_dataset("parquet",
                  data_files=["hf://datasets/jxie/coco_captions@a2ed90d49b61dd13dd71f399c70f5feb897f8bec/data/train-00011-of-00182-f499a31d286235db.parquet",])
# df = load_dataset("jxie/coco_captions", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)-00011-of-00182-f499a31d286235db.parquet:   0%|          | 0.00/114M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
prompt = """I have a caption and I want to make it slightly simpler (w.r.t background and colors) and specific, and turn it a caption that describes the same thing
If the caption is not appropriate to be converted, respond with “NO”. For example:

1. "A dog rolling in the snow at sunset" → "A dog rolling in the snow"
2. "pink photo of Tokyo" → "buildings in Tokyo"
3. "Anti-fracking protest rocks NY governor's state of the state address" → "group of people protesting in front of a house"
4. "st peter's square: St Peters Square in Rome Italy" → "a square with the colosseum in the background"
5. "A Queen Elizabeth II Prince Andrew On The Balcony Of Buckingham Palace After Trooping The Colour Ceremony. 1962." → "A Queen on a balcony"
6. "Two little Chihuahua puppies for sale" → "Two little Chihuahua puppies"
7. "Evaluate and Adjust Your Safeguards Evaluate and adjust safeguards and practices in light of results of: System testing and monitoring. Material chang" → "NO"
8. "how-to-watch-the-kitten-bowl" → "A kitten drinking out of a bowl"
9. "Voters head to polls for municipal elections across Ontario" → "A person walking and there is a sign that says vote here"
10. "44 Romantic Barn Wedding Lights Ideas" → "A man and woman dancing at a wedding"
11. "martini drink isolated on white flat vector image" → "A martini drink on white background"
12. "Selway High-Back Executive Chair" → "A high-back chair"
13. "Nickelodeon Paw Patrol'Calling All Pups' Soft Potty Seat" → "NO"
14. "Young people skateboarding on city streets" → "Young skateboarders with a building in the background"
15. "Alternate view of the Sterling Silver Chihuahua Bead Charm by The Black Bow Jewelry Co." → "NO"
16. "A woman cutting a large white sheet cake" → "A woman cutting a large cake."



do this for the following caption: "{context}".

Format your response as
```
Step 1: <simplified>
```
"""

In [ ]:
len(ds["train"]["caption"])

3114

In [ ]:
print(ds['train']['filename'])

['COCO_val2014_000000326564.jpg', 'COCO_val2014_000000326564.jpg', 'COCO_val2014_000000326564.jpg', 'COCO_val2014_000000326564.jpg', 'COCO_val2014_000000187286.jpg', 'COCO_val2014_000000187286.jpg', 'COCO_val2014_000000187286.jpg', 'COCO_val2014_000000187286.jpg', 'COCO_val2014_000000187286.jpg', 'COCO_val2014_000000032760.jpg', 'COCO_val2014_000000032760.jpg', 'COCO_val2014_000000032760.jpg', 'COCO_val2014_000000032760.jpg', 'COCO_val2014_000000032760.jpg', 'COCO_val2014_000000156754.jpg', 'COCO_val2014_000000156754.jpg', 'COCO_val2014_000000156754.jpg', 'COCO_val2014_000000156754.jpg', 'COCO_val2014_000000156754.jpg', 'COCO_val2014_000000089556.jpg', 'COCO_val2014_000000089556.jpg', 'COCO_val2014_000000089556.jpg', 'COCO_val2014_000000089556.jpg', 'COCO_val2014_000000089556.jpg', 'COCO_val2014_000000301634.jpg', 'COCO_val2014_000000301634.jpg', 'COCO_val2014_000000301634.jpg', 'COCO_val2014_000000301634.jpg', 'COCO_val2014_000000301634.jpg', 'COCO_val2014_000000266206.jpg', 'COCO_val

In [ ]:
client = openai.OpenAI()
def generate(content):
    messages=[
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": content},
                  {
                      "type": "image_url",
                      "image_url": {
                          "url": "https://i.postimg.cc/QCnfP8xh/image-1.jpg",
                      }
                  },
              ],
          }
      ]
    print("running generate")
    model = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=1,
        messages=messages,
        max_tokens=1000,
    )
    new_thought = model.choices[0].message.content

    return new_thought

In [ ]:
file_path = "DALL-E-2025-01-02-15-40-13-An-abstract-black-and-white-silhouette-of-a-woman-cutting-a-large-cake.webp"

In [ ]:
from tqdm import tqdm

all_responses = []
curr_filename = ''
max_caption = ''
count = 0

# Create a progress bar
total_iterations = len(ds["train"]["filename"])
with tqdm(total=total_iterations, desc="Processing") as pbar:
    for filename, caption in zip(ds["train"]["filename"], ds["train"]["caption"]):
        if curr_filename != filename and curr_filename != '':
            try:
                response = generate(prompt.format(context=max_caption))
                print(f"{max_caption=}\n{response}")
                print()
                all_responses.append((max_caption, response))
                count += 1
            except Exception as e:
                print(f"Error processing {curr_filename}: {e}")

            curr_filename = filename
            max_caption = caption
        elif curr_filename != filename and curr_filename == '':
            curr_filename = filename
            max_caption = caption
        else:
            max_caption = max_caption if len(max_caption) > len(caption) else caption

        # Update the progress bar
        pbar.update(1)


Processing:   0%|          | 0/3114 [00:00<?, ?it/s]

running generate


Processing:   0%|          | 3/3114 [00:02<43:17,  1.20it/s]

max_caption='some baseball players are playing baseball on a field'
```
Step 1: Some players are playing baseball on a field.
```

running generate


Processing:   0%|          | 8/3114 [00:04<26:59,  1.92it/s]

...

max_caption='A skier on skies and a snowboarder carrying a snowboard on a mountain slope.'
```
Step 1: A skier and a snowboarder on a mountain slope.
```

running generate


Processing: 100%|█████████▉| 3106/3114 [23:36<00:03,  2.42it/s]

max_caption='A bearded skateboarder rolls down a slope at night'
```
Step 1: A skateboarder rolling down a slope
```

running generate


Processing: 100%|██████████| 3114/3114 [23:38<00:00,  2.20it/s]

max_caption='A person is standing alongside a deep pathway through a meadow with trees.'
```
Step 1: A person standing by a pathway in a meadow.
```



In [ ]:
print(len(all_responses))

621


In [ ]:
import pandas as pd
import os

# Assuming all_responses is a list of tuples [(caption, response), ...]
# Extract both caption and response
all_responses_processed = [(caption, response.strip('`\n.').split(": ", 1)[-1].lower())
                           for caption, response in all_responses]

# Create DataFrame with both "Caption" and "Response" columns
df = pd.DataFrame(all_responses_processed, columns=["Caption", "Response"])

# Specify the directory where you want to save the file
directory = "/content/drive/MyDrive/excel_result_simplified_only/excel_captions"
os.makedirs(directory, exist_ok=True)  # Create the directory if it doesn't exist

# Save the file in the specified directory
file_path = os.path.join(directory, "train-00011-of-00182-f499a31d286235db.parquet.xlsx")
df.to_excel(file_path, index=False)

print(f"Excel file saved to: {file_path}")

Excel file saved to: /content/drive/MyDrive/excel_result_simplified_only/excel_captions/train-00011-of-00182-f499a31d286235db.parquet.xlsx
